In [1]:
import time
import random

In [2]:
import pandas as pd
import numpy as np

In [3]:
from dask.distributed import Client, progress
client = Client(threads_per_worker=4, n_workers=1)
client

Client Scheduler: tcp://127.0.0.1:45315 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 1 Cores: 4 Memory: 32.60 GB


In [4]:
def costly_simulation(list_param):
    time.sleep(1)
    return sum(list_param)

In [5]:
%time costly_simulation([1, 2, 3, 4])

CPU times: user 9.85 ms, sys: 5.68 ms, total: 15.5 ms
Wall time: 1 s


10

In [6]:
input_params = pd.DataFrame(
    np.random.random(size=(240, 4)),
    columns=['param_a', 'param_b', 'param_c', 'param_d']
)
input_params.head()

,param_a,param_b,param_c,param_d
0,0.510807,0.363127,0.093281,0.313247
1,0.882044,0.799111,0.927558,0.827475
2,0.888158,0.317091,0.939675,0.528515
3,0.856787,0.045295,0.497535,0.757286
4,0.128956,0.126547,0.984538,0.415079


In [13]:
# futures = []
# for parameters in input_params.values:
#     future = client.submit(costly_simulation, parameters)
#     futures.append(future)

futures = client.map(costly_simulation, input_params.to_numpy())

In [14]:
results = client.gather(futures)

In [15]:
results[:5]

[1.280463024712415,
 3.436188226893662,
 2.6734390806300663,
 2.15690232696733,
 1.6551200674849267]

In [12]:
futures = client.map(costly_simulation, input_params.values)

In [14]:
results = client.gather(futures)
len(results)

500

<hr>

Trying Dask on the SPV sim!

In [1]:
import sys
import os
import numpy as np
import pandas as pd

from tqdm import tqdm
import time
import numba
import datetime

In [2]:
os.chdir("/home/ubuntu/git/")

vor_path = "/home/ubuntu/git/active_vertex"
# vor_path = 'C:\\Users\\Pranav\\git\\active_vertex'
sys.path.append(vor_path)

from voronoi_model.voronoi_model_periodic import Tissue

In [3]:
from dask.distributed import Client, progress
client = Client(threads_per_worker=4, n_workers=1)
client

Client Scheduler: tcp://127.0.0.1:43739 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 1 Cores: 4 Memory: 32.60 GB


In [80]:
# @numba.njit
# def get_D_eff(X0, Xmax, L, tmax, v0, Dr, n=19):
    
#     X0_norm = X0 - L/2
#     center_cells = np.argsort(np.sqrt(X0_norm[:, 0]**2 + X0_norm[:, 1]**2))[:n]
    
#     dX = np.fmod(Xmax[center_cells] - X0[center_cells], L/2)
#     dr2 = np.zeros(n, dtype=np.float32)
#     for i in range(n):
#         dr2[i] = dX[i, 0] ** 2 + dX[i, 1] ** 2
    
#     Ds = dr2.mean() / (4*tmax)
#     D0 = v0**2/(2*Dr)
#     return Ds / D0

# def vor_to_D_eff(vor, n=7):
#     return get_D_eff(
#         vor.x_save[0],
#         vor.x_save[-1],
#         vor.L,
#         vor.tfin,
#         vor.v0,
#         vor.Dr,
#         n=n,
#     )

In [4]:
# p_space = np.linspace(3.5, 3.9, 11)
# v_space = np.linspace(2e-3, 3e-2, 8)
p_space = np.array([3.8])
v_space = np.array([2e-2])
d_space = np.linspace(0.8, 2, 7)
replicates = np.arange(3, dtype=int)

param_space = np.meshgrid(
    p_space,
    v_space,
    d_space,
    replicates,
)
param_space = np.array(param_space).T.reshape(-1, 4)

In [5]:
param_space

array([[3.8 , 0.02, 0.8 , 0.  ],
       [3.8 , 0.02, 1.  , 0.  ],
       [3.8 , 0.02, 1.2 , 0.  ],
       [3.8 , 0.02, 1.4 , 0.  ],
       [3.8 , 0.02, 1.6 , 0.  ],
       [3.8 , 0.02, 1.8 , 0.  ],
       [3.8 , 0.02, 2.  , 0.  ],
       [3.8 , 0.02, 0.8 , 1.  ],
       [3.8 , 0.02, 1.  , 1.  ],
       [3.8 , 0.02, 1.2 , 1.  ],
       [3.8 , 0.02, 1.4 , 1.  ],
       [3.8 , 0.02, 1.6 , 1.  ],
       [3.8 , 0.02, 1.8 , 1.  ],
       [3.8 , 0.02, 2.  , 1.  ],
       [3.8 , 0.02, 0.8 , 2.  ],
       [3.8 , 0.02, 1.  , 2.  ],
       [3.8 , 0.02, 1.2 , 2.  ],
       [3.8 , 0.02, 1.4 , 2.  ],
       [3.8 , 0.02, 1.6 , 2.  ],
       [3.8 , 0.02, 1.8 , 2.  ],
       [3.8 , 0.02, 2.  , 2.  ]])

In [29]:
f = 200
t0 = 0
tmax = 0.1
dt = 0.025
n_t = int((tmax - t0) * f / dt) + 1  # calculates the n_t to get the desired dt

# a = 0.4
k = 2
J = 0.

n_c_norm = 235

In [30]:
to_dir = "/home/ubuntu/git/evomorph/data/2020-09-29_AWS_dasktest/"

if not os.path.exists(to_dir):
    os.mkdir(to_dir)

In [31]:
def pcounter(n):
    i = 1
    while True:
        yield i/n
        i += 1

cores = 8
gen = pcounter((param_space.shape[0] // cores) + 1)
def count():
    return next(gen)

In [32]:
def costly_simulation(list_param):
    time.sleep(1)
    return sum(list_param)

In [44]:
def simulate(
    params, 
#     progress_bar=False, 
#     print_updates=False,
):
    p, v, d, rep = params
    prefix = f"p0{p:.2f}_v0{v:.2e}_reldens{d:.2f}_rep{int(rep)}"
    
    vor = Tissue()
    vor.make_init2(L=10, n_c=int(n_c_norm * d))
    
    vor.set_GRN_t_span(t0, tmax, n_t, scaling_factor=f);
    vor.v0 = v
    
#     vor.n_warmup_steps = int(150 / dt)
    vor.n_warmup_steps = 1

    W = J * np.array([[1, 0], [0, 1]])
    vor.set_interaction(W=W, pE=0)
    vor.A0 = vor.L**2 / n_c_norm
    vor.P0 = p * np.sqrt(vor.A0)
    
    vor.Dr = 0.01
    vor.kappa_A = 0.2
    vor.kappa_P = 0.1
    vor.a = vor.A0 / 2
    vor.k = k
    
    start_time = time.time()
    vor.simulate(progress_bar=False, print_updates=False);
    end_time = time.time()
    
    secs_elapsed = int(end_time) - int(start_time)
    mins_elapsed = secs_elapsed / 60
    it_per_sec = vor.n_t / secs_elapsed
    
    fname = os.path.abspath(os.path.join(to_dir, prefix))

    np.savez_compressed(fname, vor.x)
    
#     print(f"Thread {count()*100:.2f}% complete")
    
    return prefix, mins_elapsed, it_per_sec
#     return prefix, [vor_to_D_eff(vor, n) for n in (7, 19, 37)]


In [34]:
futures = client.map(simulate, param_space[:4])

In [19]:
results = client.gather(futures)

In [35]:
client.close()

distributed.nanny - WARNING - Worker process still alive after 3 seconds, killing


In [24]:
results

[('p03.80_v02.00e-02_reldens0.80_rep0', 0.45, 29.666666666666668),
 ('p03.80_v02.00e-02_reldens1.00_rep0', 0.6166666666666667, 21.64864864864865),
 ('p03.80_v02.00e-02_reldens1.20_rep0', 0.7, 19.071428571428573),
 ('p03.80_v02.00e-02_reldens1.40_rep0', 0.7666666666666667, 17.41304347826087)]

In [22]:
from glob import glob

In [23]:
glob(to_dir + '*')

['/home/ubuntu/git/evomorph/data/2020-09-29_AWS_dasktest/p03.80_v02.00e-02_reldens1.40_rep0.npz',
 '/home/ubuntu/git/evomorph/data/2020-09-29_AWS_dasktest/p03.80_v02.00e-02_reldens0.80_rep0.npz',
 '/home/ubuntu/git/evomorph/data/2020-09-29_AWS_dasktest/p03.80_v02.00e-02_reldens1.20_rep0.npz',
 '/home/ubuntu/git/evomorph/data/2020-09-29_AWS_dasktest/p03.80_v02.00e-02_reldens1.00_rep0.npz']